ЛАБОРАТОРНАЯ РАБОТА 5.2


In [16]:
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import sys

# согласованная матричная норма
def matrix_norm(matrix):
  sum = 0
  for i in range(len(matrix)):
    sum += abs(matrix[i])
  return max(sum)

# согласованная векторная норма
def vec_norm(v):
  return max(map(abs, v))

# генерация вектора
def generate_vec(l, r, n):
  vec = np.random.uniform(l, r, n)
  return vec

# генерация матрицы
def generate_matrix(l, r, n):
    a = np.random.uniform(l, r, (n, n))
    return a

def increase_diag_elems(a, diag):
    n = len(a)
    for i in range(0, len(a)):
        a[i][i] = diag * sum(abs(a[i][j]) if j != i else 0 for j in range(n))
    return a

# Вычисляем диагональное преобладание
# Проверить выполнение условия диагонального преобладания
def calc_diagonal_dominance(a):
  degree = max(abs(a[i][i]) - sum(abs(a[i][j]) if j != i else 0 for j in range(len(a))) for i in range(len(a)))
  return degree > 0


метод Якоби

In [6]:
def jacobi(A, f):
  eps = 1e-6
  n = len(A)
  D = np.array([[0 if i != j else A[i][j] for j in range(n)] for i in range(n)])
  D_inv = np.linalg.inv(D)
  P = np.dot(-D_inv, A - D)

  norm_P = matrix_norm(deepcopy(P))
  q = np.max(np.abs(np.linalg.eigvals(P)))
  if not (norm_P < 1):
    print('||P|| ≤ q < 1 not working')
    print(f'norma_P: {norm_P}')
  else:
    print('||P|| ≤ q < 1 correct')
    print(f'norma_P: {norm_P}')

  g = np.dot(D_inv, f)
  x_k = g
  iters = 1
  while True:
    x_ki = np.dot(P, x_k) + g
    # критерий остановки итерационного процесс используя равномерную норму
    if vec_norm(x_ki - x_k) < eps:
      break
    iters += 1
    x_k = x_ki
  return x_ki, iters


Метод Зейделя

In [38]:
def zeidel(A, f):
  eps = 1e-6
  n = len(A)
  D = np.array([[0 if i != j else A[i][j] for j in range(n)] for i in range(n)])
  A_n = np.array([[0 if i <= j else - A[i][j] / A[i][i] for j in range(n)] for i in range(n)])
  A_v = np.array([[0 if i >= j else - A[i][j] / A[i][i] for j in range(n)] for i in range(n)])
  D_inv = np.linalg.inv(D)

  x_k = np.dot(D_inv, f)
  iters = 1
  while True:
    x_ki = np.zeros(len(x_k))
    for i in range(n):
      for j in range(n):
        x_ki += A_n[i][j] * x_ki[j] + A_v[i][j] * x_k[j]
      x_ki[i] += f[i] / A[i][i]
    iters += 1
    if vec_norm(x_ki - x_k) < eps:
      break
    x_k = x_ki
  return x_ki, iters

тестирование

In [39]:
n = 100
a = generate_matrix(1, 10, n)
a = increase_diag_elems(a, 3)
print('matrix a:')
print(a)
f = generate_vec(1, 10, n)
print(f'\nvector f: {f}')

diag_cond = calc_diagonal_dominance(a)
print(f'\ncheck for diagonal dominance condition: {diag_cond}')

# correct res
x = np.dot(np.linalg.inv(a), f)

# jacobi res
x_j, iters = jacobi(a, f)

# zeidel res
x_z, itrs = zeidel(a, f)

print(f'\ncorrect res: {x}')
print(f'\njacobi res: {x_j}')
print(f'\nzeidel res: {x_z}')

print('\n\nmethods iterations:')
from tabulate import tabulate
mydata = [[iters, itrs]]

# create header
head = ["Jacobi", "Zeidel"]

# display table
print(tabulate(mydata, headers=head, tablefmt="grid"))

print(f'Jacobi iters / Zeidels iters: {iters/itrs}')

matrix a:
[[1673.18664163    3.07723018    6.03498596 ...    2.57782977
     6.46011845    9.68410145]
 [   5.1816453  1480.56248826    6.58798962 ...    9.89730148
     2.49833286    7.97657581]
 [   9.34244779    3.28564968 1699.18516292 ...    4.44822118
     5.67437181    7.40581673]
 ...
 [   8.00333499    4.22485228    6.14148132 ... 1661.03524786
     7.27109037    2.5497818 ]
 [   5.81438184    2.54261082    7.77094685 ...    2.33122965
  1555.34222955    5.48337809]
 [   4.93183909    8.17853633    7.3228769  ...    9.86657969
     8.47476648 1642.47757091]]

vector f: [2.47373341 8.91390561 2.83005682 7.7974442  7.50794229 6.79914262
 6.4019982  3.97645116 9.38201179 8.45999941 7.0516114  6.63535218
 1.72923426 1.79583406 3.33875492 6.57751383 8.98793205 1.34318547
 4.53886833 5.34536071 7.28619954 5.94489864 5.38877705 1.93635144
 4.28145559 6.00721705 7.43897371 8.7611063  4.7559474  7.86204664
 5.7123703  2.4554749  9.52722514 4.39443217 3.92712822 9.24566901
 2.82605378 7